In [2]:
# Package control

from langchain.agents import initialize_agent
from langchain_core.tools import BaseTool
from langchain_experimental.tools.python.tool import PythonREPLTool
from pandas import read_csv, to_numeric
from langchain_community.llms import LlamaCpp  # Use this for GGUF models
from langchain.agents.output_parsers import ToolsAgentOutputParser
import pandas as pd
from langchain.callbacks import StdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackHandler
from llama_cpp import Llama

In [ ]:

# Load locally existing model to llamaccp compiler
# Model in binary format - ccp decompresses
llm = LlamaCpp(
   # model_path="./models/phi-2-gguf/phi-2.Q4_K_M.gguf",  # quantized GGUF model
    model_path= "./models/zephyr-7b-beta-gguf/zephyr-7b-beta.Q4_K_M.gguf",
    max_new_tokens=500,
    n_ctx=4096,
    verbose=True
)


/home/joonas/.pyenv/versions/3.11.13/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3639: UserWarning: WARNING! max_new_tokens is not default parameter.
                max_new_tokens was transferred to model_kwargs.
                Please confirm that max_new_tokens is what you intended.
  if await self.run_code(code, result, async_=asy):
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ./models/zephyr-7b-beta-gguf/zephyr-7b-beta.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = huggingfaceh4_zephyr-7b-beta
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embeddi

llama_model_loader: - kv  13:                      tokenizer.ggml.tokens arr[str,32000]   = ["<unk>", "<s>", "</s>", "<0x00>", "<...
llama_model_loader: - kv  14:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  15:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  16:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  17:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  18:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  19:            tokenizer.ggml.padding_token_id u32              = 2
llama_model_loader: - kv  20:               general.quantization_version u32              = 2
llama_model_loader: - type  f32:   65 tensors
llama_model_loader: - type q4_K:  193 tensors
llama_model_loader: - type q6_K:   33 t

In [21]:
print(f"n_ctx: {llm.n_ctx}")          # context length
print(f"n_threads: {llm.n_threads}")  # threads used

n_ctx: 4096
n_threads: None


In [ ]:
# Create tooling

class GetPreview(BaseTool):
    name: str = "GetPreview"
    description: str = "Previvews a file"

    def _run(self, file_path: str):
        file_path = file_path.strip()
        if not file_path.lower().endswith(".csv"):
            raise ValueError("Expected a .csv file.")
        df = read_csv(file_path)
        print(df.shape[0])
        return {
            f'''Preview of the file: - Rows: {df.shape[0]} - Columns: {df.shape[1]}- Column names: {list(df.columns)}'''
        }



g_prev = GetPreview()
# List tools for model
tools = [ 
        g_prev,
         PythonREPLTool()
         ]

prefix = """
You are a tool-using data analyst for sales. 
You work with CSV files using Python and pandas.

Follow these steps exactly:

1. ALWAYS preview the file before getting an answer.
2. Determine if the answer requires code:
   - If yes, generate it in one line and execute it using the appropriate tool.
   - If no, do not generate code, just return the answer.
3. Use only the outputs from tools; do not invent numbers or data.
4. Return a concise human-readable result in **1–2 sentences**.
5. Never include intermediate reasoning in the final output.
"""
suffix = """
When you have the Final Answer:

- Include only the final result from the tool outputs.
- Add at most one short sentence of context or observation.
- Do NOT include code or reasoning steps unless specifically requested.

Question: {question}
{agent_scratchpad}
"""
# Initialize / Create agent
agent = initialize_agent(
    tools=tools, 
    llm=llm, 
    type="zero-shot-react-description",
    verbose=True,
    max_iterations=3,
    agent_kwargs={
        "prefix": prefix,
        "suffix": suffix,
    },
    callbacks=[StdOutCallbackHandler(),
                ]
    )


/tmp/ipykernel_7745/451130424.py:88: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [5]:
from langchain_experimental.agents import create_csv_agent
agent_2 = create_csv_agent(
    llm,
    "sales042019.csv",        # your CSV file
    verbose=True,
    allow_dangerous_code=True,  # let it run pandas safely
    prefix="""
You are a data analyst.
You can only use the tool `python_repl_ast` to run pandas/numpy code.
Never invent new tool names.
Always answer questions by:
1. Thinking
2. Calling `python_repl_ast`
3. Returning the result clearly.
"""
)

In [15]:
print(agent.agent.llm_chain.prompt.template)


You are a tool-using data analyst for sales. 
You work with CSV files using Python and pandas.

Follow these steps exactly:

1. ALWAYS preview the file before getting an answer.
2. Determine if the answer requires code:
   - If yes, generate it in one line and execute it using the appropriate tool.
   - If no, do not generate code, just return the answer.
3. Use only the outputs from tools; do not invent numbers or data.
4. Return a concise human-readable result in **1–2 sentences**.
5. Never include intermediate reasoning in the final output.


GetPreview - Previvews a file
Python_REPL - A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [GetPreview, Python_REPL]
Action Input: the input to the 

In [14]:
# Input question with prompt
user_question = f'What are the top 5 most sold products in the file sales042019.csv'
#Shortly describe the file sales042019.csv

# Run agent
response = agent.run("Using the file sales042019.csv. What would you do to find out outliers in the data.")
print(response)




> Entering new AgentExecutor chain...


Llama.generate: 394 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   16935.15 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    4892.20 ms /    41 runs   (  119.32 ms per token,     8.38 tokens per second)
llama_perf_context_print:       total time =    4914.34 ms /    42 tokens
Llama.generate: 394 prefix-match hit, remaining 109 prompt tokens to eval


Thought: First let's preview the file to see if it meets our requirements.

Action: GetPreview
Action Input: sales042019.csv18289

Observation: {"Preview of the file: - Rows: 18289 - Columns: 8- Column names: ['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date', 'Purchase Address', 'OrderDt', 'DateOfOrder']"}
Thought:

llama_perf_context_print:        load time =   16935.15 ms
llama_perf_context_print: prompt eval time =    3556.76 ms /   109 tokens (   32.63 ms per token,    30.65 tokens per second)
llama_perf_context_print:        eval time =   15113.51 ms /   125 runs   (  120.91 ms per token,     8.27 tokens per second)
llama_perf_context_print:       total time =   18746.94 ms /   234 tokens
Llama.generate: 503 prefix-match hit, remaining 273 prompt tokens to eval



Action: Python_REPL
Action Input: import pandas as pd; df = pd.read_csv('sales042019.csv', index_col=False); q1 = df['Price Each'].quantile(0.25); q3 = df['Price Each'].quantile(0.75); print('Outlier Range: ' + str((q1 - 1.5 * (q3 - q1)) + ' to ' + str(q3 + 1.5 * (q3 - q1)))
Observation: SyntaxError("'(' was never closed", ('<string>', 1, 157, "import pandas as pd; df = pd.read_csv('sales042019.csv', index_col=False); q1 = df['Price Each'].quantile(0.25); q3 = df['Price Each'].quantile(0.75); print('Outlier Range: ' + str((q1 - 1.5 * (q3 - q1)) + ' to ' + str(q3 + 1.5 * (q3 - q1)))\n", 1, 0))
Thought:

llama_perf_context_print:        load time =   16935.15 ms
llama_perf_context_print: prompt eval time =   35078.50 ms /   273 tokens (  128.49 ms per token,     7.78 tokens per second)
llama_perf_context_print:        eval time =   16178.00 ms /   132 runs   (  122.56 ms per token,     8.16 tokens per second)
llama_perf_context_print:       total time =   51335.37 ms /   405 tokens


Action: Python_REPL
Action Input: import pandas as pd; df = pd.read_csv('sales042019.csv', index_col=False); q1 = df['Price Each'].quantile(0.25).values[0]; q3 = df['Price Each'].quantile(0.75).values[0]; print('Outlier Range: ' + str((q1 - 1.5 * (q3 - q1)) + ' to ' + str(q3 + 1.5 * (q3 - q1)))
Observation: SyntaxError("'(' was never closed", ('<string>', 1, 177, "import pandas as pd; df = pd.read_csv('sales042019.csv', index_col=False); q1 = df['Price Each'].quantile(0.25).values[0]; q3 = df['Price Each'].quantile(0.75).values[0]; print('Outlier Range: ' + str((q1 - 1.5 * (q3 - q1)) + ' to ' + str(q3 + 1.5 * (q3 - q1)))\n", 1, 0))
Thought:

> Finished chain.
Agent stopped due to iteration limit or time limit.


In [7]:
import pandas as pd
#df = pd.read_csv('sales042019.csv'); 
#df['DateOfOrder'] = pd.to_datetime(df['DateOfOrder'])
#df = df[(df['DateOfOrder'].dt.month == 4) & (df['DateOfOrder'].dt.year == 2019)]
#df.groupby('Product')['Quantity Ordered'].sum().reset_index().sort_values(by='Quantity Ordered',ascending=False).head(5)

#df = pd.read_csv('sales042019.csv'); 
#df['Order Date'] = pd.to_datetime(df['Order Date']); 
#df['Month'] = df['Order Date'].dt.month; 
#df['Year'] = df['Order Date'].dt.year; df_April = df[df['Month'] == 4]; 
#df_April.groupby('Product')['Quantity Ordered'].sum().reset_index(name='Total Quantity').sort_values('Total Quantity', ascending=False).head(5)
#sales = pd.read_csv('sales042019.csv')
#sales.groupby('DateOfOrder')['Quantity Ordered'].sum().sort_values(ascending=False).head()
#sales['Total'] = sales['Quantity Ordered'] * sales['Price Each']
#grouped = sales.groupby(['Product'])['Total'].sum()
#sorted_results = grouped.reset_index().sort_values('Total', ascending=False)
#sorted_results

#df.sort_values('Total Sales', ascending=True).head(5)

#print(pd.read_csv('sales042019.csv').groupby(['DateOfOrder'])['Quantity Ordered'].sum().sort_values(ascending=False).head(5))
print(pd.read_csv('sales042019.csv').isnull().sum())


Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
OrderDt             0
DateOfOrder         0
dtype: int64
